In [ ]:

# The MIT License (MIT) Copyright (c) 2024 Andrea Andrés Urbano & Luis Axel Núñez Quintana 
#
# Permission is hereby granted, free of charge, to any person obtaining a copy of
# this software and associated documentation files (the "Software"), to deal in the Software without
# restriction, including without limitation the rights to use, copy, modify, merge, publish,
# distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in all copies or
# substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,
# INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
# NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES
# OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/LuisAxel/NLP-PerceiverAR/blob/main/traductor.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    Run in Google Colab</a>
  </td>
</table>

# NLP Programa 3: Perceiver AR
-------
Integrantes:
- Andrés Urbano Andrea
- Núñez Quintana Luis Axel

## 0.- Imports

In [ ]:
!pip install keras_core

In [ ]:
from collections import Counter
import keras_core as keras
import matplotlib.pyplot as plt
import nltk
import os
import pandas as pd
import pathlib
import random
from sklearn.decomposition import PCA
import string
import tensorflow as tf
import time
import torch
from torch import optim
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import vocab as Vocab
import warnings

Using TensorFlow backend


In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Disable tensorflow debugging logs
os.environ["KERAS_BACKEND"] = "torch"

In [ ]:
torch.__version__
torch.manual_seed(77)

In [ ]:
# Disable warnings
warnings.filterwarnings("ignore")

## 1.- Conjuntos de entrenamiento y validación

In [ ]:
def download_text_pairs():
    path_to_zip = tf.keras.utils.get_file(
        'spa-eng.zip',
        origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
        extract=True)
    path_to_file = pathlib.Path(path_to_zip).parent/'spa-eng/spa.txt'

    with open(path_to_file) as f:
        lines = f.read().split("\n")[:-1]

    text_pairs = []
    for line in lines:
        eng, spa = line.lower().split("\t")
        text_pairs.append((eng, spa))
    return text_pairs

In [ ]:
def split_text_pairs(text_pairs, val_percentage = 0.005, random_seed=43):
    random.Random(random_seed).shuffle(text_pairs)
    num_val_samples = int(val_percentage * len(text_pairs))
    num_train_samples = len(text_pairs) - num_val_samples
    train_pairs = text_pairs[:num_train_samples]
    val_pairs = text_pairs[num_train_samples:]
    return train_pairs, val_pairs

In [ ]:
def merge_pairs(text_pairs):
    return [eng + ' ' + spa  for eng, spa in text_pairs]

In [ ]:
text_pairs = download_text_pairs()
train_pairs, val_pairs = split_text_pairs(text_pairs)
test_pairs = val_pairs

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")

for s in train_pairs[:3]:
    print(s)


118964 total pairs
118370 training pairs
594 validation pairs
('the old woman fell and could not get up.', 'la anciana se cayó y no pudo levantarse.')
('what is this the abbreviation for?', '¿de qué es abreviatura esto?')
("you're not sick.", 'no estás enferma.')


## 2.- Pipeline

- Crea vocabulario y define tokenizers.

In [ ]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 86.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
spa_tokenizer = get_tokenizer('spacy', language='es_core_news_sm')

In [ ]:
def build_vocab(text, tokenizer):
    counter = Counter()
    for eng, spa in text:
        counter.update(tokenizer(eng))
        counter.update(tokenizer(spa))
    return Vocab(counter, specials=['<unk>', '<pad>', '<eos>', '<bos>'])

spa_vocab = build_vocab(train_pairs + val_pairs, spa_tokenizer)
spa_vocab.set_default_index(37546) # evita error <ukn>

In [ ]:
spa_vocab_size = len(spa_vocab)
print(f'Vocab sizes: Spanish - {spa_vocab_size}')

Vocab sizes: Spanish - 38433


In [ ]:
maxlen = 64

def data_process(text, vocab, tokenizer):
    data = []
    for eng, spa in text:
        tensor_eng = torch.tensor([vocab[token] for token in tokenizer(eng)],
                                dtype=torch.long)
        tensor_spa = torch.tensor([vocab[token] for token in tokenizer(spa)],
                                dtype=torch.long)
        if tensor_eng.shape[0] < maxlen - 2 and tensor_spa.shape[0] < maxlen - 2: #We are adding <bos> and <eos>
            x = tensor_eng[:]
            y = tensor_spa[:]
            data.append((x, y))
    return data

train_data = data_process(train_pairs, spa_vocab, spa_tokenizer)
val_data = data_process(val_pairs, spa_vocab, spa_tokenizer)

In [ ]:
print(f'train data size: {len(train_data)}, val data size: {len(val_data)}')
print(train_data[0])

train data size: 118370, val data size: 594
(tensor([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13]), tensor([14, 15, 16, 17, 18, 19, 20, 21, 13]))


In [ ]:
batch_size = 128
PAD_IDX = spa_vocab['<pad>']
EOS_IDX = spa_vocab['<eos>']
BOS_IDX = spa_vocab['<bos>']


def maxlen_pad(tensor):
    return tensor if tensor.size(1) == maxlen else torch.cat([tensor, torch.full((tensor.size(0), maxlen - tensor.size(1)), PAD_IDX, dtype=torch.long)], dim=1)

def generate_batch(data_batch):
    eng, spa = [], []
    for (eng_item, spa_item) in data_batch:
        eng.append(eng_item)
        spa.append(torch.cat([torch.tensor([BOS_IDX]),
                              spa_item,
                              torch.tensor([EOS_IDX])], dim=0))

    eng = pad_sequence(eng, batch_first=True, padding_value=PAD_IDX)
    spa = pad_sequence(spa, batch_first=True, padding_value=PAD_IDX)

    eng = maxlen_pad(eng)
    spa = maxlen_pad(spa)

    eng_spa = torch.stack([torch.cat([eng[i], spa[i]], dim=0) for i in range(eng.shape[0])])

    spa = spa[:, 1:]
    spa = maxlen_pad(spa)

    return eng_spa, spa


train_loader = DataLoader(train_data, batch_size=batch_size,
                          shuffle=True, collate_fn=generate_batch,
                          num_workers=4, pin_memory=True)

val_loader = DataLoader(val_data, batch_size=batch_size,
                        shuffle=True, collate_fn=generate_batch,
                        num_workers=4, pin_memory=True)

test_loader = DataLoader(val_data, batch_size=batch_size,
                        shuffle=True, collate_fn=generate_batch,
                        num_workers=4, pin_memory=True)

In [ ]:
%%timeit
train_batch, target_batch = next(iter(train_loader))

516 ms ± 12.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
train_batch, target_batch = next(iter(train_loader))

In [ ]:
train_batch.shape, target_batch.shape

(torch.Size([128, 128]), torch.Size([128, 64]))

In [ ]:
train_batch[:1], target_batch[:1]

(tensor([[1063,  608, 7542,  349,   89,   27,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    3,   28,   30,   31,  947, 5244,  880,   29,
            92,   27,    2,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
             1,    1,    1,    1,    1,    1,    1,    1]]),
 tensor([[  28,   30,   31,  947, 5244,  880,   29,   92,   27,    2,    1,    1,
             1,    1,    1,    1,    

## 3.- Modelo

In [ ]:
class CrossAttention(nn.Module):
    def __init__(self, dim, maxlen, n_heads=16, bias=True):
        super().__init__()
        self.n_heads = n_heads
        self.scale = (dim // n_heads) ** -0.5       # 1/sqrt(d)
        self.q = nn.Linear(dim, dim, bias = bias)
        self.k = nn.Linear(dim, dim, bias = bias)
        self.v = nn.Linear(dim, dim, bias = bias)

        self.o = nn.Linear(dim, dim, bias = bias)

        self.register_buffer("bias", torch.tril(torch.ones(maxlen, maxlen * 2)).view(1, 1, maxlen, maxlen * 2))

    def forward(self, kv, q):
        B, L_kv, D_kv = kv.shape
        B, L_q,  D_q = q.shape

        q = self.q(q)
        k = self.k(kv)
        v = self.v(kv)

        q = torch.reshape(q, [B, L_q, self.n_heads, -1])     # B, L_q,  nh,  i
        q = torch.permute(q, [0, 2, 1, 3])                   # B, nh,   L_q, i

        k = torch.reshape(k, [B, L_kv, self.n_heads, -1])    # B, L_kv, nh,  i
        k = torch.permute(k, [0, 2, 3, 1])                   # B, nh,   i,   L_kv

        v = torch.reshape(v, [B, L_kv, self.n_heads, -1])    # B, L_kv, nh,   i
        v = torch.permute(v, [0, 2, 1, 3])                   # B, nh,   L_kv, i

        qk = torch.matmul(q, k) * self.scale                 #(B, nh, L_q, i)(B, nh, i, L_kv)
                                                             # B, nh, L_q, L_kv

        qk = qk.masked_fill(self.bias[:,:,:L_q,:L_kv] == 0, float('-inf'))

        attn = torch.softmax(qk, dim=-1)

        v_attn = torch.matmul(attn, v)                       #(B, nh, L_q, L_kv)(B, nh, L_kv, i)
                                                             # B, nh, L_q, i
        v_attn = torch.permute(v_attn, [0, 2, 1, 3])         # B, L_q, nh, i
        v_attn = torch.reshape(v_attn, [B, L_q, D_q])        # B, L_q, D_q

        x = self.o(v_attn)
        return x


In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, dim, maxlen, n_heads=16, bias=True):
        super().__init__()
        self.n_heads = n_heads
        self.scale = (dim // n_heads) ** -0.5
        self.qw = nn.Linear(dim, dim, bias = bias)
        self.kw = nn.Linear(dim, dim, bias = bias)
        self.vw = nn.Linear(dim, dim, bias = bias)

        self.ow = nn.Linear(dim, dim, bias = bias)
        self.register_buffer("bias", torch.tril(torch.ones(maxlen, maxlen)).view(1, 1, maxlen, maxlen))

    def forward(self, x):
        B, L, D = x.shape
        q = self.qw(x)
        k = self.kw(x)
        v = self.vw(x)

        B, L, D = q.shape
        q = torch.reshape(q, [B, L, self.n_heads, -1])
        q = torch.permute(q, [0, 2, 1, 3])
        k = torch.reshape(k, [B, L, self.n_heads, -1])
        k = torch.permute(k, [0, 2, 3, 1])
        v = torch.reshape(v, [B, L, self.n_heads, -1])
        v = torch.permute(v, [0, 2, 1, 3])

        qk = torch.matmul(q, k) * self.scale
        qk = qk.masked_fill(self.bias[:,:,:L,:L] == 0, float('-inf'))

        attn = torch.softmax(qk, dim=-1)

        v_attn = torch.matmul(attn, v)
        v_attn = torch.permute(v_attn, [0, 2, 1, 3])
        v_attn = torch.reshape(v_attn, [B, L, D])

        x = self.ow(v_attn)
        return x

In [ ]:
class CrossTransformer(nn.Module):
    def __init__(self, dim, maxlen, heads=16, mlp_dim=4096, rate=0.1):
        super().__init__()

        self.ln_1 = nn.LayerNorm(dim)

        self.attn = CrossAttention(dim, maxlen)

        self.ln_2 = nn.LayerNorm(dim)

        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.ReLU(),
            nn.Dropout(rate),
            nn.Linear(mlp_dim, dim),
            nn.Dropout(rate),
        )

    def forward(self, kv, q):
        x = self.attn(self.ln_1(kv), self.ln_1(q)) + q
        return self.mlp(self.ln_2(x)) + x


In [ ]:
class SelfTransformer(nn.Module):
    def __init__(self, dim, maxlen, heads=16, mlp_dim=4096, rate=0.0):
        super().__init__()
        self.ln_1 = nn.LayerNorm(dim)
        self.attn = SelfAttention(dim, maxlen)
        self.ln_2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.ReLU(),
            nn.Dropout(rate),
            nn.Linear(mlp_dim, dim),
            nn.Dropout(rate),
        )

    def forward(self, x):
        x = self.attn(self.ln_1(x)) + x
        return self.mlp(self.ln_2(x)) + x

In [ ]:
class PerceiverAR(nn.Module):
    def __init__(self, input_dim, vocab_size, maxlen, depth=5,
                 mlp_dim=4096, rate=0.2):
        super().__init__()

        # num latents = spanish len
        self.latent_dim = maxlen

        self.embedding = nn.Embedding(vocab_size, input_dim)
        self.pos_embedding = nn.Parameter(
            torch.randn(1, maxlen * 2, input_dim))

        self.cross_attn = CrossTransformer(input_dim, maxlen)

        self.transformer = nn.Sequential()

        self.transformer = nn.ModuleList([
            SelfTransformer(input_dim, maxlen) for _ in range(depth)
        ])

        self.head = nn.Linear(input_dim, vocab_size, bias=False)

    def forward(self, x):

        B, L = x.shape

        x = self.embedding(x)
        x += self.pos_embedding[:, :L]

        y = x[:, self.latent_dim:]

        x = self.cross_attn(kv = x, q = y)

        for layer in self.transformer:
            x = layer(x)

        x = self.head(x)
        return x


model_dim = 1024
depth = 5
mlp_dim = 4096

perceiver = PerceiverAR(input_dim=model_dim, vocab_size=spa_vocab_size,
          maxlen=maxlen, depth=depth, mlp_dim=mlp_dim)

output = perceiver(train_batch)
output.shape, target_batch.shape

(torch.Size([128, 64, 38433]), torch.Size([128, 64]))

## 4.- Entrenamiento

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

perceiver.to(device)

cuda:0


PerceiverAR(
  (embedding): Embedding(38433, 1024)
  (cross_attn): CrossTransformer(
    (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (attn): CrossAttention(
      (q): Linear(in_features=1024, out_features=1024, bias=True)
      (k): Linear(in_features=1024, out_features=1024, bias=True)
      (v): Linear(in_features=1024, out_features=1024, bias=True)
      (o): Linear(in_features=1024, out_features=1024, bias=True)
    )
    (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (mlp): Sequential(
      (0): Linear(in_features=1024, out_features=4096, bias=True)
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
      (3): Linear(in_features=4096, out_features=1024, bias=True)
      (4): Dropout(p=0.1, inplace=False)
    )
  )
  (transformer): ModuleList(
    (0-4): 5 x SelfTransformer(
      (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (attn): SelfAttention(
        (qw): Linear(in_features=1024, out_features=1024, b

In [ ]:
PAD_IDX = spa_vocab.get_stoi()['<pad>']
EOS_IDX = spa_vocab.get_stoi()['<eos>']
BOS_IDX = spa_vocab.get_stoi()['<bos>']

PAD_IDX, EOS_IDX, BOS_IDX

(1, 2, 3)

In [ ]:
optimizer = optim.Adam(perceiver.parameters(), lr=3e-4)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=PAD_IDX)

In [ ]:
def train(model, device, train_loader, optimizer, epoch):
    start = time.time()
    running_loss = 0.0
    model.train()
    for inputs, targets in train_loader:
        targets = targets.view(-1)
        inputs, targets = inputs.to(device), targets.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        outputs = outputs.view(-1, outputs.size(-1))
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'\nTime for epoch {epoch} is {time.time()-start:4f} sec Train loss: {running_loss / len(train_loader):4f}')

In [ ]:
def translate(model, sentence, device, maxlen, vocab, tokenizer):
    with torch.no_grad():
        model.eval()

        # Sentence to tokens
        idx = torch.tensor([vocab[token] for token in tokenizer(sentence)],
                                    dtype=torch.long)

        # Add padding to maxlen
        idx = torch.cat([idx, torch.ones(maxlen - idx.size(0), dtype=torch.int64)])

        # Add <BOS>
        idx = torch.cat([idx, torch.tensor([BOS_IDX])])

        idx = idx.reshape([1, -1])

        # Start predicting spanish sentence
        for _ in range(maxlen):

            idx = idx.to(device)

            logits = model(idx)[:, -1, :]
            probs = torch.softmax(logits, dim=-1)

            _, idx_next = torch.topk(probs, k=1, dim=-1)
            idx = torch.cat((idx, idx_next), dim=1)

        txt = ' '.join([vocab.get_itos()[_] for _ in idx[0]])

    # Cut generation on <eos> and remove <pad> and <bos>
    return txt.split("<eos>")[0].replace(" <pad>", "").replace(" <bos>", "")

sentences = ['i hate raining days.',
             'i dislike the heat.',
             'i like apples.']

for s in sentences:
    trans = translate(perceiver, s, device, maxlen, spa_vocab, spa_tokenizer)
    print(f"\n{trans}")


i hate raining days . paced cepillas bouquet whirl reelected irán songs sobornar pañal anonadada coñac actualizaré worse evitaron resistido rewound ten-minute predominates búfalos gag decírselo sudor yukichi cedí imitate invertebrados revisas poop llamadme directamente dumbfounded stand author's provocando gastamos afeitándote cacharon rabo debiste exportaciones crece sospechoso dissatisfaction devuélveselo cautivó drastically desmentido cocidas hablamos mitología echaré spraying satélites oiría plana respuestas calceta castiguen backseat zealand extrovertidos juntarte reinició rifles

i dislike the heat . paced incomoda handout élite pundonoroso moverme bife sobornar selle involucrar partículas irish retirados jodiendo kushikatsu asistió belgium desvistiendo profunda percibido cussing nicely babeó pianos crucé acabe peer pasajero 1173 rítmicos percibirse postal abundan d. accent chaotic hablases run fuentes barrita avanzó bigot dark-green perdiste bunches confrontado garage visitado 

In [ ]:
load = True

if load:
    perceiver.load_state_dict(torch.load('./pesos.weights.pt'))
else:
  epochs = 10

  for epoch in range(epochs):
      train(perceiver, device, train_loader, optimizer, epoch)

      # Translate test sentences
      for s in sentences:
          trans = translate(perceiver, s, device, maxlen, spa_vocab, spa_tokenizer)
          print(trans)

## 5.- Evaluación (BLEU)

In [ ]:
def bleu_example():
    # Lista de oraciones de referencia (lista de listas)
    referencias = [['El', 'gato', 'está', 'en', 'la', 'alfombra'],
                   ['El', 'perro', 'juega', 'en', 'el', 'parque'],
                   ['El', 'cielo', 'está', 'despejado'],
                   ['El', 'sol', 'brilla', 'intensamente'],
                   ['Los', 'pájaros', 'cantan', 'en', 'los', 'árboles']]

    # Lista de oraciones candidatas (lista de listas)
    candidatas = [['El', 'gato', 'está', 'durmiendo', 'en', 'la', 'alfombra'],
                  ['El', 'perro', 'juega', 'en', 'el', 'jardín'],
                  ['El', 'cielo', 'está', 'soleado'],
                  ['El', 'sol', 'brilla', 'intensamente'],
                  ['Los', 'pájaros', 'trinan', 'en', 'los', 'árboles']]

    # Calcular el BLEU score para cada oración candidata
    for i in range(len(candidatas)):
        referencia = referencias[i]
        candidata = candidatas[i]

        bleu_score = nltk.translate.bleu_score.sentence_bleu([referencia], candidata)
        print(f"BLEU score para la oración {i+1}: {bleu_score}")

In [ ]:
bleu_example()

BLEU score para la oración 1: 8.44484326442819e-78
BLEU score para la oración 2: 0.7598356856515925
BLEU score para la oración 3: 8.636168555094496e-78
BLEU score para la oración 4: 1.0
BLEU score para la oración 5: 7.262123179505913e-78


In [ ]:
def format_string(s):
  # Remove special characters
  s = s.translate(str.maketrans('', '', string.punctuation + '¡¿'))
  # Delete multiple spaces
  return ' '.join(s.split())

In [ ]:
def bleu_eval(test_data):

  # Divide pairs in input/target
  input = [s for s, _ in test_data]
  target = [t for _, t in test_data]

  # Get model outputs
  output = []
  for s in input:
    trans = translate(perceiver, s, device, maxlen, spa_vocab, spa_tokenizer)
    output.append(trans)

  # Delete multiple spaces and special characters
  input = [format_string(s) for s in input]
  target = [format_string(s) for s in target]
  output = [format_string(s) for s in output]

  # Delete input part from model output
  for i in range(0, len(output)):
    output[i] = output[i][len(input[i]) + 1:]

  # Make list of lists for BLEU
  target = [s.split() for s in target]
  output = [s.split() for s in output]

  # Compute BLEU
  score = 0
  for i in range(len(output)):
    t = target[i]
    o = output[i]

    bleu_score = nltk.translate.bleu_score.sentence_bleu([t], o)
    score += bleu_score

  print(f'BLEU score promedio: {score/len(output)}')

bleu_eval(test_pairs)

BLEU score promedio: 0.07805201475749886


In [ ]:
# Save weights:
#torch.save(perceiver.state_dict(), './pesos.weights.pt')

# **Perceiver AR**

## **1. Preprocesamiento de datos**

Utilizamos un conjunto de datos que contiene oraciones en inglés y con su respectiva traducción en español. Preprocesamos los datos de tal forma que una lista tiene dos elementos: una oración en inglés y otra en español, formando parejas de valores por elemento de la lista.

## **2. Pipeline**

### <font color='coral'>2.1 Vocabulario</font>

En el pipeline formamos un vocabulario en español apartir de las oraciones de entrada. Y además, agregamos a este vocabulario las palabras especiales '\<unk>', '\<pad>', '\<eos>', '\<bos>'.

### <font color='coral'>2.2 Tokenización</font>

Posteriormente, para cada conjunto de parejas en español e inglés, las convertimos a un tensor a partir del vocalulario creado y tokenizando cada palabra. Y utilizando un maxlen = 64, lo que indica que  cada oración debe tener una longitud de 64 palabras.

### <font color='coral'>2.3. Lotes</font>

Utilizamos lotes de 128 y estos fueron construidos agregando las palabras especiales cada tensor generado anteriormente. Un tensor con palabras especiales queda de la siguiente manera:

```
[tensor.oracion_ingles + tensor.PAD_IDX + tensor.BOS_IDX + tensor.oracion_español + tensor.EOS_IDX + tensor.PAD_IDX]
```

Solamente se definen lotes para entrenamiento (train_batch) y validación (target_batch). Dado que el tamaño de lo lotes es 128 y el maxlen de 64, se consigue el siguiente tamaño de lotes.

```
train_batch.shape  --> ([128, 128])
target_batch.shape --> ([128, 64])
```

Puesto que train batch esta compuesto por oracioness en inglés y español (64*2) y el target batch de solo oraciones en español (64).


## **3. Arquitectura**

<p align="center">
  <img src="https://raw.githubusercontent.com/Andrea585976/cuantica/main/PerceiverAR.png" alt="Perceiver AR" width="320" height="550">
</p>


Imagen tomada de Hawthorne, C., Jaegle, A., Cangea, C., Borgeaud, S., Nash, C., Malinowski, M., Dieleman, S., Vinyals, O., Botvinick, M. (14 de junio de 2022). General-purpose, long-context autoregressive modeling with Perceiver AR. Cornell University. pp. 2.


### <font color='coral'>3.1 Cross/Self Attention</font>

En el modelo de la arquitectura se definió una clase para el Cross Attention y el Self Attention por separado. Además, se crearon otras clases que implementan el Transformer con su respectimas Atenciones.  

### <font color='coral'>3.2 Arquitectura del Perceiver AR</font>

### <font color='cornflowerblue'>3.2.2 Embedding y latents</font>

En la arquitectura del Perceiver AR se crea en primer lugar los embeddings de la entrada, después se dividen los latentes de acuerdo a la siguiente regla:
```
y = x[:, self.latent_dim:]
```
Donde y es la oración en español, x el arreglo compuesto por la oración en inglés y español y latent_dim = maxlen. De este modo y contiene las oraciones con las palabras a predecir en español, puesto que Perceiver AR es un modelo Autoregresivo.


### <font color='cornflowerblue'>3.2.3 Cross Attention Layer</font>

Posteriormente, se aplica un Transformer que implementa el Cross Attention con máscara, para evitar ver los valores futuros. Ésta es una máscara triangular y la definimos como
```
qk = qk.masked_fill(self.bias[:,:,:L_q,:L_kv] == 0, float('-inf'))
```

En la capa de Cross Attention los valores Q, K junto con V, son diferentes, $$X_Q \not= X_{KV}$$ Cross Attention es usado para reducir la entrada.


### <font color='cornflowerblue'>3.2.4 Self Attention Layer</font>

Enseguida, se aplica varias capas de Transformer que implementan el Self Attention decuerdo a la profundidad del modelo.

En la capa de Self Attention los valores Q, K junto con V, son iguales, $$X_Q = X_{KV}$$ Self Attention es usado para mantener la forma la entrada.

Y finalmente en la salida se aplica una capa Linear.

## **4. Selección de hiperparámetros**

Para seleccionar los hiperparámetros del modelo se realizaron varias pruebas modificando el learning rate, el número de capas de self attention, el número de cabezas en la Atención, la dimensión del MLP.

Varios de estos parámetros se escogieron el primer lugar de acuerdo a los valores con los que fueron probados en el paper original, dado que estos eran valores ya comprobados.

Sin embargo, como sabemos la selección de hiperparámetros es empírica, por lo que decidimos probar con varios al azar hasta obtener el mejor valor de BLUE.


## **5. Conclusiones**

Perceiver AR es una arquitectura relativamente nueva por lo que no hay mucha información sobre ella, más que en papers oficiales y páginas sobre investigación en machine learning. Esta fue una de las razones por las cuales la implementación del Perceiver llevo su tiempo para poder programarlo, debido a que hay que realizar une etapa de investigación previa para poder entender el modelo.

Otra de las dificultades con la que nos enfrentamos fue el tamaño de los lotes de entrenamiento y validación, ya que durante el modelo de la aquitectura se realizan operaciones con matrices que en varias ocasiones son redimensionadas para poder realizar operaciones matriciales.

El Perceiver es una arquitectura que al igual que el Transformer se basa en atenciones, pero a diferencia del Transformer, el Perceiver utiliza dos tipos de atenciones: la atención cruzada y la autoatención. La diferencia entre este tipo de atenciones es que la atención cruzada utiliza valores de Q diferentes a K y V, mientras que en la autoatención los valores de Q, K y V son iguales.

Apesar de que el modelo no llega al BLEU propuesto de 0.19, sí llega a un BLEU de aproximadamente 0.07-0.09. Como pudimos observar en las traducciones generadas por el modelo, se lograron traducir muy bien algunas oraciones, pero para otras se inventaba palabras o no traducía bien una palabra a su correspondiente en español; observamos que los detalles anteriores impactan significativamente en el score de BLUE, por lo que también lograr llegar a valor solicitado es un reto.